## Проект "Авиарейсы без потерь"

### Задание 4.1
1 point possible (graded)
База данных содержит список аэропортов практически всех крупных городов России. В большинстве городов есть только один аэропорт. Исключение составляет:

In [ ]:
# answer count(a.airport_name)
query = "
select a.city, count(a.airport_name)
from dst_project.airports as a
group by a.city
having count(a.airport_name)>1 
"

### Задание 4.2
##### Вопрос 1. 
Таблица рейсов содержит всю информацию о прошлых, текущих и запланированных рейсах. Сколько всего статусов для рейсов определено в таблице?

In [ ]:
# answer 6
query = "
select count(distinct f.status)
from dst_project.flights as f
"

##### Вопрос 2. 
Какое количество самолетов находятся в воздухе на момент среза в базе (статус рейса «самолёт уже вылетел и находится в воздухе»).

In [ ]:
# answer 58
query = "
select count (distinct f.flight_id)
from dst_project.flights as f
where f.status = 'Departed'
"

##### Вопрос 3. 
Места определяют схему салона каждой модели. Сколько мест имеет самолет модели  (Boeing 777-300)?

In [ ]:
# answer 402
query = "
select count(s.seat_no) 
from dst_project.seats as s join dst_project.aircrafts a on s.aircraft_code = a.aircraft_code
where a.model = 'Boeing 777-300'
"

##### Вопрос 4. 
Сколько состоявшихся (фактических) рейсов было совершено между 1 апреля 2017 года и 1 сентября 2017 года?
Здесь и далее состоявшийся рейс означает, что он не отменён, и самолёт прибыл в пункт назначения.

In [ ]:
# answer 74227
query = "
select count(f.flight_id) 
from dst_project.flights as f
where  (f.actual_arrival between '2017-04-01'::date  and  '2017-09-01'::date)
   and (f.status = 'Arrived') and (f.status != 'Cancelled')
"

### Задание 4.3
##### Вопрос 1.  
Сколько всего рейсов было отменено по данным базы?

In [ ]:
# answer 437
query = "
select count(f.flight_id) 
from dst_project.flights as f
where  f.status = 'Cancelled'
"

##### Вопрос 2. 
Сколько самолетов моделей типа Boeing, Sukhoi Superjet, Airbus находится в базе авиаперевозок?

In [ ]:
# answer 1 + 3 + 3

query = "
select count(*)
from dst_project.aircrafts as a
where a.model like '%Sukhoi Superjet%'
"
query = "
select count(*)
from dst_project.aircrafts as a
where a.model like '%Boeing%'
"
query = "
select count(*)
from dst_project.aircrafts as a
where a.model like '%Airbus%'
"

##### Вопрос 3. 
В какой части (частях) света находится больше аэропортов?

In [ ]:
# answer Europe, Asia
query = "
with count_countinent as (
    select 'ASIA', count(a.timezone)
    from dst_project.airports as a
    where a.timezone like 'Asia%'
    union
    select 'Europe', count(a.timezone)
    from dst_project.airports as a
    where a.timezone like 'Europe%'
    union
    select 'Australia', count(a.timezone)
    from dst_project.airports as a
    where a.timezone like 'Australia%'
    union
    select 'Europe, Asia', count(a.timezone)
    from dst_project.airports as a
    where a.timezone like 'Europe%' OR a.timezone like 'Asia%'

)
select * from count_countinent
order by 2 desc
limit 1

##### Вопрос 4. 
У какого рейса была самая большая задержка прибытия за все время сбора данных? Введите id рейса (flight_id).

In [ ]:
# answer 157571
query = "
select t_delta.flight_id from (
select f.flight_id, f.actual_arrival,f.scheduled_arrival, f.actual_arrival - f.scheduled_arrival as delay_time
from dst_project.flights as f
WHERE actual_arrival IS NOT NULL
order by delay_time desc
limit 1
) t_delta
"

### Задание 4.4
##### Вопрос 1.
Когда был запланирован самый первый вылет, сохраненный в базе данных?

In [ ]:
# август 14, 2016, 11:45 вечера
query = "
select f.scheduled_departure
from dst_project.flights as f
order by 1
limit 1
"

#### Вопрос 2. 
Сколько минут составляет запланированное время полета в самом длительном рейсе?

In [ ]:
# answer - 530 минут
query = "
SELECT date_part('hour', f.scheduled_arrival - f.scheduled_departure) * 60 +
       date_part('minute', f.scheduled_arrival - f.scheduled_departure) flight_time
from dst_project.flights as f
order by 1 desc
limit 1
"

##### Вопрос 3. 
Между какими аэропортами пролегает самый длительный по времени запланированный рейс?

In [ ]:
# answer - DME - UUS
query = "
SELECT f.departure_airport, f.arrival_airport, f.scheduled_arrival - f.scheduled_departure
from dst_project.flights as f
where f.scheduled_arrival - f.scheduled_departure in (
        SELECT f.scheduled_arrival - f.scheduled_departure max_flight_time
        from dst_project.flights as f
         order by 1 desc
        limit 1)
group by f.departure_airport, f.arrival_airport, f.scheduled_arrival - f.scheduled_departure
order by 2 desc
"

##### Вопрос 4. 
Сколько составляет средняя дальность полета среди всех самолетов в минутах? Секунды округляются в меньшую сторону (отбрасываются до минут).

In [ ]:
# answer - 128 min
query = "
SELECT date_part('hour', avg(f.actual_arrival - f.actual_departure)) * 60 +
         date_part('minute', avg(f.actual_arrival - f.actual_departure)) avg_atual_time_distance
from dst_project.flights as f
"

### Задание 4.5
##### Вопрос 1.
Мест какого класса у SU9 больше всего?

In [ ]:
# answer - Economy
query = "
SELECT s.fare_conditions, count(s.fare_conditions)
from dst_project.seats as s
where s.aircraft_code = 'SU9' 
group by s.fare_conditions
order by 2 desc
limit 1
"

##### Вопрос 2. 
Какую самую минимальную стоимость составило бронирование за всю историю?

In [ ]:
# answer - 1308700
query = "
SELECT max(b.total_amount)
from dst_project.bookings b
"

##### Вопрос 3. 
Какой номер места был у пассажира с id = 4313 788533?

In [ ]:
# answer - 2A
query = "
SELECT bp.seat_no
from dst_project.tickets t
    join dst_project.boarding_passes bp
    on t.ticket_no = bp.ticket_no
where t.passenger_id = '4313 788533'

### Задание 5.1
#####  Вопрос 1. 
Анапа — курортный город на юге России. Сколько рейсов прибыло в Анапу за 2017 год

In [ ]:
# answer - 486
query = "
SELECT count(f.flight_id)
from dst_project.flights f 
    join dst_project.airports a
    on f.arrival_airport = a.airport_code
where a.city = 'Anapa' and f.status = 'Arrived' and date_part('year', f.actual_arrival) = '2017'
"

##### Вопрос 2. 
Сколько рейсов из Анапы вылетело зимой 2017 года?

In [ ]:
# answer - 127 
query = "
SELECT count(f.flight_id)
from dst_project.flights f 
WHERE (f.departure_airport = 'AAQ')
    AND (date_part('year', f.actual_departure) = 2017)
    AND (date_part('month', f.actual_departure) IN (12, 1, 2))
"

##### Вопрос 3. 
Посчитайте количество отмененных рейсов из Анапы за все время.

In [ ]:
# answer - 1
query = "
SELECT count(f.flight_id)
from dst_project.flights f 
WHERE (f.departure_airport = 'AAQ')
  AND (f.status = 'Cancelled')
"

##### Вопрос 4. 
Сколько рейсов из Анапы не летают в Москву?

In [ ]:
# answer - 453
query = "
SELECT count(f.flight_no)
from dst_project.flights f 
    join dst_project.airports a on f.arrival_airport = a.airport_code
WHERE (f.departure_airport = 'AAQ')
  AND (a.city != 'Moscow')
"    

##### Вопрос 5. 
Какая модель самолета летящего на рейсах из Анапы имеет больше всего мест?

In [ ]:
# answer - 737
query = "
SELECT f.aircraft_code, count(distinct s.seat_no)
from dst_project.flights f 
    join dst_project.seats s on f.aircraft_code = s.aircraft_code
WHERE f.departure_airport = 'AAQ'
group by f.aircraft_code

## Финальный проект

In [ ]:
-- выборка по количеству выкупенных билетов на рейсы
SELECT 
    distinct f.flight_id, f.status,
    count(tf.ticket_no)
FROM dst_project.flights f
    left join dst_project.ticket_flights tf on f.flight_id = tf.flight_id
WHERE f.departure_airport = 'AAQ'
  AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
  AND f.status not in ('Cancelled')
group by f.flight_id, f.status
order by 3

In [ ]:
# общий запрос
query = "
with 
tf_anapa as -- выбор стоимости только по рейсам Анапа-зима ts
  (
  select tf.flight_id, tf.fare_conditions, sum(tf.amount) fare_conditions_amount, count(tf.ticket_no) fare_conditions_tickets
  from dst_project.ticket_flights tf
  where tf.flight_id in 
    (
    SELECT distinct(flight_id)
    FROM dst_project.flights f
    WHERE f.departure_airport = 'AAQ'
  AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
  AND f.status not in ('Cancelled')
    )
  group by tf.flight_id, tf.fare_conditions
  ),
  
sn as 
    (
    SELECT
    tf_anapa.flight_id,
    tf_anapa.fare_conditions,
    count(s.seat_no) total_seats_no
    FROM tf_anapa 
        left join dst_project.flights f on tf_anapa.flight_id = f.flight_id 
        left join dst_project.seats s on f.aircraft_code = s.aircraft_code and tf_anapa.fare_conditions = s.fare_conditions

    group by tf_anapa.flight_id, f.aircraft_code, tf_anapa.fare_conditions
    ), 
    
tf_anapa_business as
    (
    Select f.*, 
        tf_anapa.fare_conditions_amount business_conditions_amount, 
        tf_anapa.fare_conditions_tickets business_conditions_tickets,
        sn.total_seats_no business_total_seats_no
    from dst_project.flights f
         left join tf_anapa on f.flight_id = tf_anapa.flight_id
        left join sn on tf_anapa.flight_id = sn.flight_id and tf_anapa.fare_conditions = sn.fare_conditions
    where tf_anapa.fare_conditions = 'Business'
    ), 
    
tf_anapa_economy as
    (
    Select f.*, 
        tf_anapa.fare_conditions_amount economy_conditions_amount, 
        tf_anapa.fare_conditions_tickets economy_conditions_tickets,
        sn.total_seats_no economy_total_seats_no
    from dst_project.flights f
         left join tf_anapa on f.flight_id = tf_anapa.flight_id
        left join sn on tf_anapa.flight_id = sn.flight_id and tf_anapa.fare_conditions = sn.fare_conditions
    where tf_anapa.fare_conditions = 'Economy'
    ), 
    
tf_anapa_null as
    (
    Select f.*, 
        tf_anapa.fare_conditions_amount null_conditions_amount, 
        tf_anapa.fare_conditions_tickets null_conditions_tickets,
        sn.total_seats_no null_total_seats_no
    from dst_project.flights f
         left join tf_anapa on f.flight_id = tf_anapa.flight_id
        left join sn on tf_anapa.flight_id = sn.flight_id and tf_anapa.fare_conditions = sn.fare_conditions
    where tf_anapa.fare_conditions is null
    )   
    
SELECT f.*, 
    tf_anapa_business.business_conditions_amount, 
    tf_anapa_business.business_conditions_tickets, 
    tf_anapa_business.business_total_seats_no, 
    tf_anapa_economy.economy_conditions_amount, 
    tf_anapa_economy.economy_conditions_tickets,
    tf_anapa_economy.economy_total_seats_no,
    tf_anapa_null.null_conditions_amount, 
    tf_anapa_null.null_conditions_tickets,
    tf_anapa_null.null_total_seats_no,
    a.model,
    aa.airport_name  as arrival_airport_name, 
    aa.longitude as arrival_airport_longitude, 
    aa.latitude as arrival_airport_latitude, 
    ad.longitude as departure_airport_longitude, 
    ad.latitude as departure_airport_latitude
FROM dst_project.flights f
    left join tf_anapa_business on f.flight_id = tf_anapa_business.flight_id
    left join tf_anapa_economy on f.flight_id = tf_anapa_economy.flight_id
    left join tf_anapa_null on f.flight_id = tf_anapa_null.flight_id
    left join dst_project.aircrafts a on f.aircraft_code = a.aircraft_code
    left join dst_project.airports aa on f.arrival_airport = aa.airport_code
    left join dst_project.airports ad on f.departure_airport = ad.airport_code
# --left join sn on tf_anapa.flight_id = sn.flight_id and tf_anapa.fare_conditions = sn.fare_conditions
WHERE f.departure_airport = 'AAQ'
    AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
    AND f.status not in ('Cancelled')
order by 1; 
"

In [ ]:
# промежуточный запрос. Решила не использовать, так как не удобный формат вывода данных для анализа
query = "

with tf as -- выбор стоимости только по рейсам Анапа-зима ts
  (
  select tf.flight_id, tf.fare_conditions, sum(tf.amount) fare_conditions_amount, count(tf.ticket_no) fare_conditions_tickets
  from dst_project.ticket_flights tf
  where tf.flight_id in 
    (
    SELECT distinct(flight_id)
    FROM dst_project.flights f
    WHERE f.departure_airport = 'AAQ'
  AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
  AND f.status not in ('Cancelled')
    )
  group by tf.flight_id, tf.fare_conditions
  ),
sn as
    (
    SELECT
    tf.flight_id,
    tf.fare_conditions,
    count(s.seat_no) total_seats_no

     FROM tf
        left join dst_project.flights f on tf.flight_id = f.flight_id 
        left join dst_project.seats s on f.aircraft_code = s.aircraft_code and tf.fare_conditions = s.fare_conditions

    group by tf.flight_id, f.aircraft_code, tf.fare_conditions
    )
    SELECT f.*, 
    tf.fare_conditions, 
    tf.fare_conditions_amount, 
    tf.fare_conditions_tickets,
    sn.total_seats_no,
    a.model,
    aa.airport_name  as arrival_airport_name, 
    aa.longitude as arrival_airport_longitude, 
    aa.latitude as arrival_airport_latitude, 
    ad.longitude as departure_airport_longitude, 
    ad.latitude as departure_airport_latitude
FROM dst_project.flights f
 left join tf on f.flight_id = tf.flight_id
 left join dst_project.aircrafts a on f.aircraft_code = a.aircraft_code
 left join dst_project.airports aa on f.arrival_airport = aa.airport_code
 left join dst_project.airports ad on f.departure_airport = ad.airport_code
 left join sn on tf.flight_id = sn.flight_id and tf.fare_conditions = sn.fare_conditions
WHERE f.departure_airport = 'AAQ'
  AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
  AND f.status not in ('Cancelled')
order by 1;  
"